A rough copy of https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/

In [1]:
%set_env GPU=1

env: GPU=1


In [2]:
import numpy as np
from PIL import Image
from tinygrad.nn import Conv2d, BatchNorm2d, Linear, optim
from datasets import fetch_mnist
from extra.training import train, evaluate

In [3]:
class LeNet5:
    def __init__(self):
        self.conv1 = Conv2d(1, 6, 5)
        self.bn1 = BatchNorm2d(6)
        self.conv2 = Conv2d(6, 16, 5)
        self.bn2 = BatchNorm2d(16)
        self.fc1 = Linear(400, 120)
        self.fc2 = Linear(120, 84)
        self.fc3 = Linear(84, 10)
    
    def __call__(self, x):
        x = self.bn1(self.conv1(x)).relu().max_pool2d(stride=2)
        x = self.bn2(self.conv2(x)).relu().max_pool2d(stride=2)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x).relu()
        x = self.fc2(x).relu()
        x = self.fc3(x).log_softmax()
        return x

In [4]:
def transform(x):
    x = [Image.fromarray(xx).resize((32, 32)) for xx in x]
    x = np.stack([np.asarray(xx) for xx in x], axis=0)
    x = x.reshape(-1, 1, 32, 32)
    return x

In [5]:
X_train, Y_train, X_test, Y_test = fetch_mnist()
X_train = X_train.reshape(-1, 28, 28) / 255.0
X_test = X_test.reshape(-1, 28, 28) / 255.0

model = LeNet5()
optimizer = optim.AdamW(optim.get_parameters(model), lr=0.001, wd=0.0005)
train(model, X_train, Y_train, optimizer, 1000, BS=128, transform=transform)
evaluate(model, X_test, Y_test, transform=transform);

100%|█████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 117.49it/s]

test set accuracy is 0.980300
